Import Libs

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# own packages
from preprocess_data import clean_text
from preprocess_data import lemmatize_text
from preprocess_data import tfidf_vec_transform, tfidf_vec_fit_transform
from preprocess_data import count_vectorizer
from models import model_sfnn_train
from models import model_rf_train, model_naives_bayes, model_logistic_regression, model_multinominalNB_train, predict_values, model_xgboost



Function for reading the data

In [8]:
# get csv data
def read_Data(path):

    # Define column names
    column_names = ['label', 'text']

    df = pd.read_csv(path, delimiter='\t', encoding="utf-8-sig", header=None, names=column_names)
    df.columns = df.columns.str.replace('\ufeff', '')
    df['label'] = df['label'].astype(str).str.replace('\ufeff', '')
    df['label'] = df['label'].astype(int)
    
    return df



In [ ]:
## read data windows
data_train_val = read_Data(r".\data\training_data_lowercase.csv")           # used for train and val data
data_test = read_Data(r".\data\testing_data_lowercase_nolabels.csv")        # data for the predictions

## read data Mac
#data_train_val = read_Data("data/training_data_lowercase.csv")           # used for train and val data
#data_test = read_Data("data/testing_data_lowercase_nolabels.csv")        # data for the predictions


## split data_train into train and val data
data_train, data_val = train_test_split(data_train_val, test_size=0.02, random_state=42, stratify=data_train_val['label'])


In [10]:
## preview data
#print(data_train.head, "\n")
#print(data_test.head, "\n")
print("Shape train data:\n", data_train_val.shape)
print("Shape test data:\n", data_test.shape)

# clean data
data_train['cleaned_text'] = data_train['text'].apply(clean_text)
data_val['cleaned_text'] = data_val['text'].apply(clean_text)
data_test['cleaned_text'] = data_test['text'].apply(clean_text)

print("Cleand text train: \n", data_train["cleaned_text"].head, "\n")
print("Cleand text test: \n", data_test["cleaned_text"].head, "\n")


## lemmatize data
data_train['lemmatized_text'] = data_train['cleaned_text'].apply(lemmatize_text)
data_val['lemmatized_text'] = data_val['cleaned_text'].apply(lemmatize_text)
data_test['lemmatized_text'] = data_test['cleaned_text'].apply(lemmatize_text)

print("lemmatized_text train: \n", data_train['lemmatized_text'].head, "\n")
print("lemmatized_text test: \n", data_test['lemmatized_text'].head, "\n")

# calc tf-idf matrix
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf_matrix_train = vectorizer.fit_transform(data_train['lemmatized_text'])
tfidf_matrix_val = vectorizer.transform(data_val['lemmatized_text'])
tfidf_matrix_test = vectorizer.transform(data_test['lemmatized_text'])

# calc tf-idf matrix with uncleand data
X_train_tfidf = vectorizer.fit_transform(data_train['text'])
X_val_tfidf = vectorizer.transform(data_val['text'])
X_test_tfidf = vectorizer.transform(data_test['text'])

#print("tfidf_matrix_train_val: \n", tfidf_matrix_train, "\n")
#print("tfidf_matrix_test: \n", tfidf_matrix_test, "\n")


# count vectorizer
#X_train_vec = count_vectorizer(data_train['lemmatized_text']).toarray()
#X_val_vec = count_vectorizer(data_val['lemmatized_text']).toarray()



Shape train data:
 (34152, 2)
Shape test data:
 (9984, 2)
Cleand text train: 
 <bound method NDFrame.head of 22543    [white, house, working, lawmakers, amending, r...
31856    [czech, police, ask, parliament, allow, prosec...
410      [equifax, tricking, consumers, waiving, right,...
11832         [delusional, democrats, get, reality, check]
20289    [grandmas, grandpas, travel, ban, states, welc...
                               ...                        
21084    [eu, ready, climate, leadership, case, us, wit...
31221    [macrons, promise, new, franceafrica, ties, ra...
883                                         [like, father]
22414    [ministers, give, mnuchin, space, define, trum...
3939     [hilariously, amazing, video, weve, waiting, e...
Name: cleaned_text, Length: 32444, dtype: object> 

Cleand text test: 
 <bound method NDFrame.head of 0       [copycat, muslim, terrorist, arrested, assault...
1       [wow, chicago, protester, caught, camera, admi...
2       [germanys, fdp, 

In [11]:
# Naive Bayes Model
model_nb = model_naives_bayes(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])


Train Accuracy:1.00

Validation Accuracy: 0.94

classification_report:
               precision    recall  f1-score   support

           0     0.9356    0.9579    0.9466       879
           1     0.9542    0.9300    0.9420       829

    accuracy                         0.9444      1708
   macro avg     0.9449    0.9440    0.9443      1708
weighted avg     0.9446    0.9444    0.9444      1708



In [ ]:
# XGBoost
model_xgboost(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])

In [ ]:
# Random Forest
model_rf = model_rf_train(tfidf_matrix_train, tfidf_matrix_val, data_train['label'], data_val['label'])

KeyboardInterrupt: 

In [ ]:
# Multinomial Naive Bayes (MultinomialNB) classifier
#model_mnb = model_multinominalNB_train(data_train['lemmatized_text'], data_val['lemmatized_text'], data_train['label'], data_val['label'])


Train Accuracy: 95.00%
Test
accuracy: 0.9334374390005856
Classification report:
               precision    recall  f1-score   support

           0       0.93      0.94      0.94      2636
           1       0.94      0.93      0.93      2487

    accuracy                           0.93      5123
   macro avg       0.93      0.93      0.93      5123
weighted avg       0.93      0.93      0.93      5123



In [ ]:
# Simple Feedforward NN -> no good results
model_sfnn_train(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])


Epoch 1/20


d:\ownCloud\Dokumente\Ironhack\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


227/227 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8548 - loss: 0.3391 - val_accuracy: 0.9295 - val_loss: 0.1874
Epoch 2/20
227/227 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9358 - loss: 0.1652 - val_accuracy: 0.9295 - val_loss: 0.1829
Epoch 3/20
227/227 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9512 - loss: 0.1287 - val_accuracy: 0.9323 - val_loss: 0.1888
Epoch 4/20
227/227 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9673 - loss: 0.0920 - val_accuracy: 0.9336 - val_loss: 0.1920
Epoch 5/20
227/227 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9785 - loss: 0.0628 - val_accuracy: 0.9362 - val_loss: 0.2145
Epoch 6/20
227/227 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9865 - loss: 0.0423 - val_accuracy: 0.9393 - val_loss: 0.2457
Epoch 7/20
227/227 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9885 - loss: 0.0343 - val_accuracy: 0.9366 - val_loss: 0.2533
908/908 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Train Accuracy: 96.39%
Validation

<Sequential name=sequential_1, built=True>

In [ ]:
model_lr = model_logistic_regression(tfidf_matrix_train, data_train['label'], tfidf_matrix_val, data_val['label'])


Train Accuracy: 96.74%

Validation Accuracy: 92.93%

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.92      0.93      2636
           1       0.92      0.94      0.93      2487

    accuracy                           0.93      5123
   macro avg       0.93      0.93      0.93      5123
weighted avg       0.93      0.93      0.93      5123



In [ ]:
## make predictions on test data
filepath = r".\data\testing_data_lowercase_labels.csv"

#predict_values(model_rf, tfidf_matrix_test, data_test, filepath)
predict_values(model_nb, tfidf_matrix_test, data_test, filepath)

✅ Predictions saved to: .\data\testing_data_lowercase_labels.csv
